In [ ]:
from keras.models import model_from_json
from neural_net import *

def warmstart(test_traj: int = 1, load_model:bool = True):
    if load_model:
        json_file = open('model.json', 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        neuralNet = model_from_json(loaded_model_json)
        loaded_model.load_weights("model.h5")
        print("Loaded model from disk")

        neuralNet.compile(loss='mean_squared_error',
                          optimizer=rms,
                          metrics=['mean_squared_error', "mean_absolute_error"])
        
    else:
        neuralNet = train_net(ntraj = 10000)
        neuralNet.compile(loss='mean_squared_error',
                          optimizer=rms,
                          metrics=['mean_squared_error', "mean_absolute_error"])
    for _ in range(test_traj):
        
        model = cartpole_model()

        x0 = [ random.uniform(0,1), random.uniform(-3.14, 3.14), random.uniform(0., 0.99)
                        , random.uniform(0., .99)]
        
        prediction = neuralNet.predict(x0)
        
        results = 

        T  = 100
        problem = crocoddyl.ShootingProblem(np.matrix(x0).T, [ model ]*T, model)
        ddp = crocoddyl.SolverDDP(problem)
        # Solving this problem
        done = ddp.solve([], [], 1000)
        del ddp.xs[0]
        
        
        

In [ ]:
from keras.models import model_from_json
from neural_net import *
import numpy as np
import random

json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
neuralNet = model_from_json(loaded_model_json)
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

neuralNet.compile(loss='mean_squared_error',
                  optimizer=rms,
                  metrics=['mean_squared_error', "mean_absolute_error"])

model = cartpole_model()

x0 = [ random.uniform(0,1), random.uniform(-3.14, 3.14), random.uniform(0., 0.99)
                , random.uniform(0., .99)]

prediction = neuralNet.predict(x0).reshape(100, 5)

##Every fifth element is control, the first 4 elements are state
ddp_xs, ddp_us = [], []
ddp_xs.append(np.matrix(x0).T)
xs = predictions[:,0:4]
us = predictions[:,4]

for _ in range(xs.shape[0]):
    ddp_xs.append(np.matrix(xs[_]).T)

for _ in  range(us.shape[0]):
ddp_us.append(np.matrix(us[_]).T)




T  = 100
problem = crocoddyl.ShootingProblem(np.matrix(x0).T, [ model ]*T, model)
ddp = crocoddyl.SolverDDP(problem)
# Solving this problem
done = ddp.solve(ddp_xs, ddp_us)

print(ddp.iter)

In [67]:

import os
from multiprocessing import Process, current_process
import numpy as np
import keras
from keras import regularizers
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import crocoddyl
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
crocoddyl.switchToNumpyArray()
import random
from model_cartpole import *

def get_trajectories(ntraj: int = 2):
    """
    In the cartpole system, each state is 4 dimensional, while the control is 1 dimensional.
    Therefore, the each row y_data should be 100 * 5 -> 500. So shape y_data : n_trjac X 500
    
    """
    initial_state = []
    trajectory = []
    for _ in range(ntraj):
        
        model = cartpole_model()

        x0 = [ random.uniform(0,1), random.uniform(-3.14, 3.14), random.uniform(0., 0.99)
                        , random.uniform(0., .99)]

        T  = 100
        problem = crocoddyl.ShootingProblem(np.matrix(x0).T, [ model ]*T, model)
        ddp = crocoddyl.SolverDDP(problem)
        # Solving this problem
        done = ddp.solve([], [], 1000)
        del ddp.xs[0]

        y = [np.append(np.array(i).T, j) for i, j in zip(ddp.xs, ddp.us)]
        initial_state.append(x0)
        trajectory.append(np.array(y).flatten())     
        
    initial_state = np.asarray(initial_state)
    trajectory = np.asarray(trajectory)
    return initial_state, trajectory

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13739861205323112508
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 10656024913558531069
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 13215368443585859412
physical_device_desc: "device: XLA_CPU device"
]


In [68]:
x, t = get_trajectories()

In [69]:
x.shape

(2, 4)

In [70]:
t.shape

(2, 500)

In [62]:
initial_state = []
trajectory = []
for _ in range(1):

    model = cartpole_model()

    x0 = [ random.uniform(0,1), random.uniform(-3.14, 3.14), random.uniform(0., 0.99)
                    , random.uniform(0., .99)]

    T  = 100
    problem = crocoddyl.ShootingProblem(np.matrix(x0).T, [ model ]*T, model)
    ddp = crocoddyl.SolverDDP(problem)
    # Solving this problem
    done = ddp.solve([], [], 1000)
    del ddp.xs[0]

    y = [np.append(np.array(i).T, j) for i, j in zip(ddp.xs, ddp.us)]
    initial_state.append(x0)
    trajectory.append(np.array(y).flatten())

initial_state = np.asarray(initial_state)
trajectory = np.asarray(trajectory)


In [63]:
initial_state

array([[0.07432075, 0.31644861, 0.79359936, 0.46079197]])

In [65]:
trajectory.shape

(1, 500)